# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
# "../../../data/LIMS_data/DDB_PD_006_AMBR_LIMS.xlsx"
experiment = 'DDB_PD_006_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Experiment Annotation,growth_rate,Feed Medium #1,...,pH Setpoint,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type
0,DDB_PD_006_AMBR_DDB25-001_R01_Main_,DDB_BM_002,R01,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air
1,DDB_PD_006_AMBR_DDB25-001_R02_Main_,DDB_BM_002,R02,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air
2,DDB_PD_006_AMBR_DDB28-001_R04_Main_,DDB_BM_002,R04,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air
3,DDB_PD_006_AMBR_DDB28-001_R05_Main_,DDB_BM_002,R05,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air
4,DDB_PD_006_AMBR_DDB29-001_R06_Main_,DDB_BM_002,R06,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air
5,DDB_PD_006_AMBR_DDB29-001_R07_Main_,DDB_BM_002,R07,AMBR 250,No,Main,DDB_PD_006_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,6.5,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Phase
0,0.000000,96.763658,6.592032,1002.967986,29.850000,1.420944,100.163720,0.0,20.95,20.617968,...,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air,batch
1,0.033333,93.658517,6.597986,999.043044,29.930144,1.459636,99.959784,0.0,20.95,20.630972,...,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air,batch
2,0.066667,92.242483,6.596841,1001.031843,29.950001,1.463318,99.903095,0.0,20.95,20.627064,...,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air,batch
3,0.100000,91.819147,6.592649,1000.072146,30.000000,1.438495,99.982168,0.0,20.95,20.617145,...,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air,batch
4,0.133333,91.436621,6.597742,999.128994,30.020000,1.572394,100.046003,0.0,20.95,20.617130,...,Ammonium hydroxide 15%,40,Stirring,Aeration,0.1,0.23,1000,4000,Air,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_006_AMBR_R01__S00,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S00,0.059732,1.0,0.256
1,DDB_PD_006_AMBR_R01__S01,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S01,3.018471,1.0,0.564
2,DDB_PD_006_AMBR_R01__S02,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S02,6.049919,2.5,1.500
3,DDB_PD_006_AMBR_R01__S03,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S03,9.018873,1.0,4.665
4,DDB_PD_006_AMBR_R01__S04,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S04,12.018302,1.0,12.360


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_006_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,2020-09-10,0.059732,DDB_PD_006_AMBR_R01__S00,succinic acid,0.000000,g/L
1,DDB_PD_006_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,2020-09-10,0.059732,DDB_PD_006_AMBR_R01__S00,ethanol,0.000000,g/L
2,DDB_PD_006_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,2020-09-10,0.059732,DDB_PD_006_AMBR_R01__S00,potassium dihydrogen phosphate,9.879014,g/L
3,DDB_PD_006_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,2020-09-10,0.059732,DDB_PD_006_AMBR_R01__S00,citric acid,0.000000,g/L
4,DDB_PD_006_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,2020-09-10,0.059732,DDB_PD_006_AMBR_R01__S00,D-glucose,20.474840,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,20.474840,0.000000,0.0,0.0,0.0,0.000000,9.879014,0.000000,0.0,DDB_PD_006_AMBR_R01__S00
1,21.585935,0.000000,0.0,0.0,0.0,0.000000,10.547562,0.000000,0.0,DDB_PD_006_AMBR_R01__S01
2,21.316798,0.162893,0.0,0.0,0.0,0.000000,10.746964,0.000000,0.0,DDB_PD_006_AMBR_R01__S02
3,18.887596,0.328206,0.0,0.0,0.0,0.000000,11.174180,0.000000,0.0,DDB_PD_006_AMBR_R01__S03
4,11.149656,0.821924,0.0,0.0,0.0,0.706556,10.444396,0.114875,0.0,DDB_PD_006_AMBR_R01__S04


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,Timepoint Sample


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L]
0,DDB_PD_006_AMBR_R01__S00,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S00,0.059732,1.0,0.256,20.474840,0.000000,0.0,0.0,0.0,0.000000,9.879014,0.000000,0.0
1,DDB_PD_006_AMBR_R01__S01,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S01,3.018471,1.0,0.564,21.585935,0.000000,0.0,0.0,0.0,0.000000,10.547562,0.000000,0.0
2,DDB_PD_006_AMBR_R01__S02,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S02,6.049919,2.5,1.500,21.316798,0.162893,0.0,0.0,0.0,0.000000,10.746964,0.000000,0.0
3,DDB_PD_006_AMBR_R01__S03,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S03,9.018873,1.0,4.665,18.887596,0.328206,0.0,0.0,0.0,0.000000,11.174180,0.000000,0.0
4,DDB_PD_006_AMBR_R01__S04,DDB_PD_006_AMBR,DDB_PD_006_AMBR_DDB25-001_R01_Main_,R01,S04,12.018302,1.0,12.360,11.149656,0.821924,0.0,0.0,0.0,0.706556,10.444396,0.114875,0.0


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)